# Модель DeepLabV3

In [1]:
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
import albumentations as A
import numpy as np
from pathlib import Path
from PIL import Image
import os
from typing import Dict, Any
import torchvision.transforms as transforms
from pytorch_lightning.callbacks import ModelCheckpoint
from typing import Dict, Any
import torchmetrics
import matplotlib.pyplot as plt
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.loggers import TensorBoardLogger

c:\ProgramData\anaconda3\envs\vkr\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\ProgramData\anaconda3\envs\vkr\Lib\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = Path(image_dir)
        self.mask_dir = Path(mask_dir)
        self.transform = transform
        self.images = sorted(list(self.image_dir.glob("*.png"))) 

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        mask_path = self.mask_dir / img_path.name  # Маска с таким же именем

        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"))  # Маска в градациях серого
        mask = (mask > 0).astype(np.float32)  # Бинарная маска (0 или 1)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image, mask = augmented["image"], augmented["mask"]
        
        if len(mask.shape) == 2:  # If mask is (256, 256), add channel dimension
            mask = mask[None, ...]  # Shape: (1, 256, 256)

        return image, mask

# Аугментация
train_transform = A.Compose([
    A.Resize(256, 256),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    A.ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(256, 256),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    A.ToTensorV2()
])

In [3]:
train_dataset = SegmentationDataset(
    image_dir=r'D:\URFU\VKR\Ind_pract\dissert\data\train\images',
    mask_dir=r'D:\URFU\VKR\Ind_pract\dissert\data\train\masks',
    transform=train_transform
)

valid_dataset = SegmentationDataset(
    image_dir=r'D:\URFU\VKR\Ind_pract\dissert\data\val\images',
    mask_dir=r'D:\URFU\VKR\Ind_pract\dissert\data\val\masks',
    transform=val_transform
)

test_dataset = SegmentationDataset(
    image_dir=r'D:\URFU\VKR\Ind_pract\dissert\data\test\images',
    mask_dir=r'D:\URFU\VKR\Ind_pract\dissert\data\test\masks',
    transform=val_transform
)

train_loader = DataLoader(train_dataset,
                                batch_size=4,
                                shuffle=True
                                )
valid_loader = DataLoader(valid_dataset,
                                batch_size=4
                                )
test_loader = DataLoader(test_dataset,
                                batch_size=4
                                )

images, masks = next(iter(train_loader))
print("Размерность изображений:", images.shape)
print("Размерность масок:", masks.shape)
print("Тип изображений:", images.dtype)
print("Тип масок:", masks.dtype)
print("Классы:", masks.unique())

Размерность изображений: torch.Size([4, 3, 256, 256])
Размерность масок: torch.Size([4, 1, 256, 256])
Тип изображений: torch.float32
Тип масок: torch.float32
Классы: tensor([0., 1.])


In [8]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
import torchmetrics
from typing import Dict, Any


class DeepLabV3PlusLightning(pl.LightningModule):
    def __init__(self, encoder_name="resnet101", learning_rate=1e-3):
        super().__init__()
        self.save_hyperparameters()
        self.model = smp.DeepLabV3Plus(
            encoder_name=encoder_name,
            encoder_weights=None,
            in_channels=3,
            classes=1,
            activation=None,
        )
        self.criterion = nn.BCEWithLogitsLoss()  # Лосс для бинарной сегментации
        self.lr = learning_rate
        
        # Инициализация метрик
        self.train_iou = torchmetrics.JaccardIndex(task="binary", num_classes=2)
        self.val_iou = torchmetrics.JaccardIndex(task="binary", num_classes=2)
        self.test_iou = torchmetrics.JaccardIndex(task="binary", num_classes=2)
        
        self.train_accuracy = torchmetrics.Accuracy(task="binary", threshold=0.5)
        self.val_accuracy = torchmetrics.Accuracy(task="binary", threshold=0.5)
        self.test_accuracy = torchmetrics.Accuracy(task="binary", threshold=0.5)
        
        self.train_precision = torchmetrics.Precision(task="binary", threshold=0.5)
        self.val_precision = torchmetrics.Precision(task="binary", threshold=0.5)
        self.test_precision = torchmetrics.Precision(task="binary", threshold=0.5)
        
        self.train_recall = torchmetrics.Recall(task="binary", threshold=0.5)
        self.val_recall = torchmetrics.Recall(task="binary", threshold=0.5)
        self.test_recall = torchmetrics.Recall(task="binary", threshold=0.5)
        
        self.train_f1 = torchmetrics.F1Score(task="binary", threshold=0.5)
        self.val_f1 = torchmetrics.F1Score(task="binary", threshold=0.5)
        self.test_f1 = torchmetrics.F1Score(task="binary", threshold=0.5)

    def forward(self, x):
        return self.model(x)
    
    def _shared_step(self, batch, stage):
        images, masks = batch
        outputs = self(images)
        loss = self.criterion(outputs, masks.float())
        
        # Применяем сигмоиду и порог для получения бинарных предсказаний
        preds = torch.sigmoid(outputs) > 0.5
        
        metrics = {
            f"{stage}_loss": loss,
            f"{stage}_iou": getattr(self, f"{stage}_iou")(preds, masks.int()),
            f"{stage}_accuracy": getattr(self, f"{stage}_accuracy")(preds, masks.int()),
            f"{stage}_precision": getattr(self, f"{stage}_precision")(preds, masks.int()),
            f"{stage}_recall": getattr(self, f"{stage}_recall")(preds, masks.int()),
            f"{stage}_f1": getattr(self, f"{stage}_f1")(preds, masks.int()),
        }
        
        # Логируем метрики
        self.log_dict(metrics, prog_bar=True if stage == "val" else False)
        
        return loss if stage != "test" else metrics

    def training_step(self, batch, batch_idx):
        return self._shared_step(batch, "train")

    def validation_step(self, batch, batch_idx):
        return self._shared_step(batch, "val")

    def test_step(self, batch, batch_idx):
        return self._shared_step(batch, "test")
    
    def on_train_epoch_end(self):
        # Сбрасываем метрики в конце эпохи
        self.train_iou.reset()
        self.train_accuracy.reset()
        self.train_precision.reset()
        self.train_recall.reset()
        self.train_f1.reset()
    
    def on_validation_epoch_end(self):
        # Сбрасываем метрики в конце эпохи
        self.val_iou.reset()
        self.val_accuracy.reset()
        self.val_precision.reset()
        self.val_recall.reset()
        self.val_f1.reset()
    
    def on_test_epoch_end(self):
        # Сбрасываем метрики в конце эпохи
        self.test_iou.reset()
        self.test_accuracy.reset()
        self.test_precision.reset()
        self.test_recall.reset()
        self.test_f1.reset()

    def configure_optimizers(self) -> Dict[str, Any]:
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, 
            patience=3, 
            factor=0.5,
            verbose=True
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",
                "interval": "epoch",
                "frequency": 1,
            },
        }

In [9]:
# Инициализация модели
model = DeepLabV3PlusLightning(encoder_name="resnet101", learning_rate=1e-3)


# Настройка логгера TensorBoard
logger = TensorBoardLogger(save_dir="../data/logs/", name="DeepLab_segmentation")

# Настройка callback для сохранения лучшей модели
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",  
    dirpath="../data/checkpoints/", 
    filename="deeplab-{epoch:02d}-{val_loss:.2f}",  
    save_top_k=1, 
    mode="min"  
)

# Инициализация тренера
trainer = pl.Trainer(
    max_epochs=50,  # Количество эпох
    accelerator="gpu" if torch.cuda.is_available() else "cpu",  # Использовать GPU, если доступно
    devices=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    log_every_n_steps=10,
)

# Обучение модели
trainer.fit(model, train_loader, valid_loader)




GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\ProgramData\anaconda3\envs\vkr\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(

   | Name            | Type               | Params | Mode 
----------------------------------------------------------------
0  | model           | DeepLabV3Plus      | 45.7 M | train
1  | criterion       | BCEWithLogitsLoss  | 0      | train
2  | train_iou       | BinaryJaccardIndex | 0      | train
3  | val_iou         | BinaryJaccardIndex | 0      | train
4  | test_iou        | BinaryJaccardIndex | 0      | train
5  | train_accuracy  | BinaryAccuracy     | 0      | train
6  | val_accuracy    | BinaryAccuracy     | 0      | train
7  | test_accuracy   | BinaryAccuracy     | 0      | train
8  | train_precision | BinaryPrecision  

c:\ProgramData\anaconda3\envs\vkr\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 0:  27%|██▋       | 293/1088 [01:44<04:44,  2.79it/s, v_num=3]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
# Тестирование модели
trainer.test(model, dataloaders=test_loader)

# Инференс

In [ ]:
# Загрузка модели из чекпоинта
model = DeepLabV3PlusLightning.load_from_checkpoint("deeplab_model.ckpt")
model.eval()

In [ ]:
import glob
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as transforms
import torch
import os
import numpy as np

# Путь к файлам
path = "/input_images"
files = glob.glob(path)
files = [file for file in files if os.path.isfile(file)]

class InferenceDataset(Dataset):
    def __init__(self, image_paths):
        self.image_paths = image_paths
        self.transform = transforms.Compose([
            transforms.Resize([256, 256]),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        image = self.transform(image)
        return image, self.image_paths[idx]  # Возвращаем также путь к файлу

# Создание DataLoader
test_dataset = InferenceDataset(files)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Инференс с использованием trainer
predictions = trainer.predict(model, dataloaders=test_loader)

# Визуализация предсказанных масок
plt.imshow(predictions[0], cmap="gray")  
plt.title("Segmentation Mask")
plt.axis("off")
plt.show()

# Создание директории для сохранения масок, если её нет
output_dir = "/output_masks"
os.makedirs(output_dir, exist_ok=True)

# Обработка предсказаний и сохранение масок
for batch, image_paths in zip(predictions, test_loader):
    pred = batch  # Предсказания для батча
    pred = torch.sigmoid(pred)
    pred = (pred > 0.5).float()
    pred = pred.squeeze().cpu().numpy()
    
    # Получаем путь к исходному изображению
    image_path = image_paths[1][0]  # Путь к файлу из датасета
    
    # Формируем имя выходного файла
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_path = os.path.join(output_dir, f"{base_name}_mask.png")
    
    # Сохранение предсказанной маски
    output_image = (pred * 255).astype(np.uint8)
    output_pil = Image.fromarray(output_image)
    output_pil.save(output_path)
    print(f"Saved mask: {output_path}")